# Seemingly Unrelated Regression
### Used when multiple equations may have correlated error terms

In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels.system import SUR

In [4]:
df = pd.read_csv('../data/sur2.csv')
df = sm.add_constant(df)
df.head()

,const,P_Eggs,Q_Eggs,P_Beer,Q_Beer,Income
0,1.0,7.03,110,8.60,152,45
1,1.0,7.38,119,8.18,160,50
2,1.0,10.40,153,9.20,119,48
3,1.0,13.58,130,8.29,154,52
4,1.0,11.69,120,10.96,146,47


In [30]:
#Define the system of equations
formulas = {
    'eq1': 'Q_Eggs ~ P_Eggs + Income + const',
    'eq2': 'Q_Beer ~ P_Beer + Income + const'
}

#Fit the SUR model
model = SUR.from_formula(formulas, data=df)
sur_results = model.fit()
sur_results.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                           System GLS Estimation Summary                           
===================================================================================
Estimator:                        GLS   Overall R-squared:                   0.1026
No. Equations.:                     2   McElroy's R-squared:                 0.1444
No. Observations:                  10   Judge's (OLS) R-squared:             0.1026
Date:                Wed, Jun 12 2024   Berndt's R-squared:                  0.2204
Time:                        14:51:35   Dhrymes's R-squared:                 0.1026
                                        Cov. Estimator:                      robust
                                        Num. Constraints:                      None
                  Equation: eq1, Dependent Variable: Q_Eggs                   
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
P_Eggs         1.8645     0.9984     1.8675     0.0618     -0.0923      3.8213
Income         0.2584     1.1570     0.2234     0.8232     -2.0092      2.5261
const          96.498     57.158     1.6883     0.0914     -15.530      208.53
                  Equation: eq2, Dependent Variable: Q_Beer                   
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
P_Beer         2.2414     4.0260     0.5567     0.5777     -5.6493      10.132
Income         0.8758     2.1900     0.3999     0.6892     -3.4165      5.1681
const          79.352     123.79     0.6410     0.5215     -163.28      321.99
==============================================================================

Covariance Estimator:
Heteroskedastic (Robust) Covariance (Debiased: False, GLS: True)
"""

In [40]:
#OLS for comparison
y_b = df['Q_Beer']
X_b = df[['P_Beer','Income']]
X_b = sm.add_constant(X_b)
ols_results_b = sm.OLS(y_b,X_b).fit()
print(ols_results_b.summary())

y_e = df['Q_Eggs']
X_e = df[['P_Eggs','Income']]
X_e = sm.add_constant(X_e)
ols_results_e = sm.OLS(y_e,X_e).fit()
print(ols_results_e.summary())

                            OLS Regression Results                            
Dep. Variable:                 Q_Beer   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                 -0.239
Method:                 Least Squares   F-statistic:                    0.1311
Date:                Wed, 12 Jun 2024   Prob (F-statistic):              0.879
Time:                        14:55:22   Log-Likelihood:                -40.878
No. Observations:                  10   AIC:                             87.76
Df Residuals:                       7   BIC:                             88.66
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         78.7664    141.926      0.555      0.5

/opt/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/opt/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [56]:
print('OLS Eggs Parameters')
print(ols_results_e.params)
print('\n')
print('OLS Beer Parameters')
print(ols_results_b.params)
print('\n')
print('SUR Parameters (Both)')
print(sur_results.params)

OLS Eggs Parameters
const     105.282710
P_Eggs      2.778056
Income     -0.120361
dtype: float64


OLS Beer Parameters
const     78.766433
P_Beer     2.289419
Income     0.878818
dtype: float64


SUR Parameters (Both)
eq1_P_Eggs     1.864506
eq1_Income     0.258434
eq1_const     96.497562
eq2_P_Beer     2.241422
eq2_Income     0.875787
eq2_const     79.352388
Name: params, dtype: float64
